In [1]:
import os
import pandas as pd
import random
import numpy
import re
from tqdm import tqdm
import json
import pickle
from concurrent.futures import ProcessPoolExecutor

from abnumber import Chain
from anarci import anarci, number

In [15]:
HEAVY_POSITIONS = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45',
       '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56',
       '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67',
       '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78',
       '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89',
       '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100',
       '101', '102', '103', '104', '105', '106', '107', '108', '109',
       '110', '111', '111A', '111B', '111C', '111D', '111E', '111F',
       '111G', '111H', '111I', '111J', '111K', '111L', '112L', '112K',
       '112J', '112I', '112H', '112G', '112F', '112E', '112D', '112C',
       '112B', '112A', '112', '113', '114', '115', '116', '117', '118',
       '119', '120', '121', '122', '123', '124', '125', '126', '127',
       '128']

HEAVY_CDR_INDEX = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                   0, 0, 0, 0, 0, 0, 0, 0, 
                   1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 
                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                   2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 
                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                   3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 
                   3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 
                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

LIGHT_POSITIONS = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45',
       '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56',
       '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67',
       '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78',
       '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89',
       '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100',
       '101', '102', '103', '104', '105', '106', '107', '108', '109',
       '110', '111', '111A', '111B', '111C', '111D', '111E', '111F',
       '112F', '112E', '112D', '112C', '112B', '112A', '112', '113',
       '114', '115', '116', '117', '118', '119', '120', '121', '122',
       '123', '124', '125', '126', '127']

LIGHT_CDR_INDEX = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                   0, 0, 0, 0, 0, 0, 0, 0, 
                   1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                   2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 
                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                   3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 
                   3, 3, 3, 3, 3, 3, 3, 3, 3, 
                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [23]:
def extract_number(s):
    return int(re.findall(r'\d+', s)[0])
# numbers = [extract_number(s) for s in LIGHT_POSITIONS]
# cdr_index = [
#     1 if 27 <= idx <= 38 else
#     2 if 56 <= idx <= 65 else
#     3 if 105 <= idx <= 117 else
#     0
#     for idx in numbers
# ]
tmp = []
for i in range(len(HEAVY_CDR_INDEX)):
    tmp.append(i)

tmp_dict = {}

for v, idx in zip(HEAVY_POSITIONS, tmp):
    tmp_dict[v] = idx
print(tmp_dict)

{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9, '11': 10, '12': 11, '13': 12, '14': 13, '15': 14, '16': 15, '17': 16, '18': 17, '19': 18, '20': 19, '21': 20, '22': 21, '23': 22, '24': 23, '25': 24, '26': 25, '27': 26, '28': 27, '29': 28, '30': 29, '31': 30, '32': 31, '33': 32, '34': 33, '35': 34, '36': 35, '37': 36, '38': 37, '39': 38, '40': 39, '41': 40, '42': 41, '43': 42, '44': 43, '45': 44, '46': 45, '47': 46, '48': 47, '49': 48, '50': 49, '51': 50, '52': 51, '53': 52, '54': 53, '55': 54, '56': 55, '57': 56, '58': 57, '59': 58, '60': 59, '61': 60, '62': 61, '63': 62, '64': 63, '65': 64, '66': 65, '67': 66, '68': 67, '69': 68, '70': 69, '71': 70, '72': 71, '73': 72, '74': 73, '75': 74, '76': 75, '77': 76, '78': 77, '79': 78, '80': 79, '81': 80, '82': 81, '83': 82, '84': 83, '85': 84, '86': 85, '87': 86, '88': 87, '89': 88, '90': 89, '91': 90, '92': 91, '93': 92, '94': 93, '95': 94, '96': 95, '97': 96, '98': 97, '99': 98, '100': 99, '101': 100, '102'

In [10]:
print(cdr_index)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [8]:
Chn_seqs = set()

In [9]:
def parse_unpair_cgz_file(path, length=149):
    """Parse the GZ-compressed CSV file."""

    # configurations
    seg_names_dict = {
        'H': ['fwh1', 'cdrh1', 'fwh2', 'cdrh2', 'fwh3', 'cdrh3', 'fwh4'],
        'K': ['fwk1', 'cdrk1', 'fwk2', 'cdrk2', 'fwk3', 'cdrk3', 'fwk4'],
        'L': ['fwl1', 'cdrl1', 'fwl2', 'cdrl2', 'fwl3', 'cdrl3', 'fwl4'],
    }

    # region_length = (26, 12, 17, 10, 38, 30, 11)

    # parse the GZ-compressed CSV file
    try:
        data_frame = pd.read_csv(path, header=1, compression='gzip')
    except EOFError:
        logging.warning('corrupted GZ-compressed CSV file: %s', path)
        return []

    # obtain a list of (CSV file name, record index, heavy chain seq., light chain seq.)-tuples
    seq_list = []
    # chn_seqs = set()  # to remove duplicated sequences
    name = os.path.basename(path).replace('.csv.gz', '')
    for idx, row_data in tqdm(data_frame.iterrows(), total=len(data_frame)):
        chain_type = row_data['locus']
        try:
            seg_names = seg_names_dict[row_data['locus']]
            chn_seq = row_data['sequence_alignment_aa']
            anarci_outputs = json.loads(row_data['ANARCI_numbering'].replace('\'', '"'))
            seg_seqs_c = [''.join(anarci_outputs[x].values()) for x in seg_names]
            assert ''.join(seg_seqs_c) in chn_seq
            if chn_seq not in Chn_seqs:
                # print(chn_seq)
                Chn_seqs.add(chn_seq)
                if '1' in [i.strip() for i in anarci_outputs[seg_names[0]].keys()]:
                    results = number(chn_seq, scheme='imgt')
                    data = results[0]
                    seq = ''.join([item[1] for item in data])
                    pattern = re.compile(r'\d+')
                    seq_index = [int(pattern.search(str(item)).group()) for item in data]
                    cdr_index = [
                        1 if 27 <= idx <= 38 else
                        2 if 56 <= idx <= 65 else
                        3 if 105 <= idx <= 117 else
                        0
                        for idx in seq_index
                    ]
                    
                    if len(seq) < length:
                        seq = seq + '-'*(length - len(seq))
                        cdr_index.extend([0]*(length-len(cdr_index)))
                    else:
                        raise AttributeError(f'Chain length more than threshold.')
    
                    # Append seq for training.
                    seq_list.append((name, seq, cdr_index, chain_type))
                    
                else:
                    raise AttributeError(f'Chain {chain_type} Length is special.')

        except AttributeError as e:
            print(f'Caught an error: {e.args[0]}')
            # print(f'Chain {chain_type} Length is special.')
            continue

    return seq_list


In [10]:
def main(chain_type='Light', num_processes=8):
    """Main entry."""

    # configurations
    root_dir = '/data/home/waitma/antibody_proj/antidiff/data/oas_unpair_human_data/'
    cgz_dpath = os.path.join(root_dir, 'unpair_select_cgzdata_200')
    save_path = os.path.join(cgz_dpath, f'{chain_type}_pad_data.pkl')
    # initialization
    # tfold_init()

    # parse all the GZ-compressed CSV files
    seq_list = []
    cgz_names = sorted(os.listdir(cgz_dpath))[:1]
    print(cgz_names)
    with ProcessPoolExecutor(max_workers=num_processes) as executor:
        results = list(tqdm(executor.map(parse_unpair_cgz_file, 
            [os.path.join(cgz_dpath, name) for name in cgz_names if chain_type in name]), total=len(cgz_names), desc=f'{chain_type}parsing GZ-compressed CSV files'))

    seq_list = []
    for result in results:
        seq_list.extend(result) 

    # Save path.
    with open(save_path, 'wb') as f:
        pickle.dump(seq_list, f)
        f.close()

In [9]:
test_save_path = './test_save.pkl'
with open(test_save_path, 'wb') as f:
    pickle.dump(seq_list, f)
    f.close()
    

In [10]:
with open(test_save_path, 'rb') as f:
    file_list = pickle.load(f)
    f.close()


In [11]:
test_seq = 'AAQPADIVLTQTPSSLPVSVGEKVTMTCKSSQTLLYSNNQKNYLAWYQQKPGQSPKLLISWAFTRKSGVPDRFTGSGSGTDFTLTIGSVKAEDLAVYYCQQYSNYPWTFGGGTRLEIKRGGGGSGGGGSGGGGSGGGGSEVQLQQSGPEVVKTGASVKISCKASGYSFTGYFINWVKKNSGKSPEWIGHISSSYATSTYNQKFKNKAAFTVDTSSSTAFMQLNSLTSEDSAVYYCVRSGNYEEYAMDYWGQGTSVTVSS'
result = number(test_seq, scheme='imgt')

In [14]:
data = result[0]

# seq = ''.join([item[1] for item in data])
# pattern = re.compile(r'\d+')
# seq_index = [int(pattern.search(str(item)).group()) for item in data]
# cdr_index = [
#     1 if 27 <= idx <= 38 else
#     2 if 56 <= idx <= 65 else
#     3 if 105 <= idx <= 117 else
#     0
#     for idx in seq_index
# ]

# if len(seq) < 149:
#     seq = seq + '-'*(149 - len(seq))
#     cdr_index.extend([0]*(149-len(cdr_index)))

In [18]:
result[1]

'L'

In [20]:
chain = Chain(test_seq, scheme='imgt')

ChainParseError: Found 2 antibody domains in sequence: "AAQPADIVLTQTPSSLPVSVGEKVTMTCKSSQTLLYSNNQKNYLAWYQQKPGQSPKLLISWAFTRKSGVPDRFTGSGSGTDFTLTIGSVKAEDLAVYYCQQYSNYPWTFGGGTRLEIKRGGGGSGGGGSGGGGSGGGGSEVQLQQSGPEVVKTGASVKISCKASGYSFTGYFINWVKKNSGKSPEWIGHISSSYATSTYNQKFKNKAAFTVDTSSSTAFMQLNSLTSEDSAVYYCVRSGNYEEYAMDYWGQGTSVTVSS"

# Need to deal the download sh file. it is impossible download all.

In [101]:
sh_fpath = '/data/home/waitma/antibody_proj/antidiff/data/oas_unpair_human_data/cgz_part_data/unpair_download_add.sh'
with open(sh_fpath, 'r') as f:
    files_list = f.readlines()

In [102]:
# 6: 1 (H:L). Select 1000 index for download.

light_index_list = []
heavy_index_list = []
non_type_index_list = []
for idx, file in enumerate(files_list):
    if 'Heavy' in file:
        heavy_index_list.append(idx)
    elif 'Light' in file:
        light_index_list.append(idx)
    else:
        non_type_index_list.append(idx)

In [103]:
sample_num = 200
random.seed(0)

light_selected_list = random.sample(light_index_list, sample_num)
heavy_selected_list = random.sample(heavy_index_list, sample_num)

In [108]:
new_download_sh_list = [files_list[i] for i in light_selected_list] + [files_list[i] for i in heavy_selected_list]
numpy.random.shuffle(new_download_sh_list)

In [109]:
sh_save_fpath = '/data/home/waitma/antibody_proj/antidiff/data/oas_unpair_human_data/cgz_part_data/unpair_download_selected_200.sh'
with open(sh_save_fpath, 'w') as f:
    for line in new_download_sh_list:
        f.write(line)

In [80]:
light_index_list[0]

100

In [81]:
files_list[100]

'wget --no-check-certificate https://opig.stats.ox.ac.uk/webapps/ngsdb/unpaired/Bender_2020/csv/ERR3664744_Light_Bulk.csv.gz\n'